In [ ]:
import pandas as pd
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt
import seaborn as sns
import theano.tensor as tt
import numpy as np
from kedro.runner import SequentialRunner
from sklearn.preprocessing import StandardScaler

%reload_kedro
%load_ext autoreload
%autoreload 2

# Analyze game data

In [ ]:
all_games = catalog.load("primary_HSNCT_all_games_2019")

### Model

In [ ]:
all_games['point_diff_per_20tuh'] = (all_games['point_diff'] / all_games['tuh']) * 20

In [ ]:
all_games.head()

In [ ]:
num_teams = len(set(all_games["team_1"].unique()) | set(all_games["team_2"].unique()))
num_rounds = all_games["round"].nunique() + 1
team_1 = all_games["team_1_index"].values
team_2 = all_games["team_2_index"].values
round_idx = all_games['round'].values
point_diff_norm = StandardScaler(with_std=False).fit_transform(all_games[["point_diff_per_20tuh"]])[:,0]

In [ ]:
# %debug
with pm.Model() as model:
    
    #hyperpriors
    sigma_strength = pm.HalfCauchy('sigma_strength', beta=20)

    # priors
    strength = pm.HalfNormal("strength", sd=sigma_strength, shape=num_teams)
    sigma_diff = pm.HalfCauchy("sigma_dfif", beta=20)
    
    # likelihood of observed data
    mu_diff = strength[team_1] - strength[team_2]
    point_diff = pm.Normal('point_diff', mu=mu_diff, sigma=sigma_diff, observed=point_diff_norm)

    trace = pm.sample(tune=2000)

In [ ]:
team_indices = (
    pd.DataFrame(list(set(all_games["team_1"].unique()) | set(all_games["team_2"].unique())))
    .reset_index()
    .rename({0: "team"}, axis=1)
    .set_index("team")
    .to_dict()["index"]
)
team_indices_reverse = {val:key for key, val in team_indices.items()}

In [ ]:
diff_summary = az.from_pymc3(trace,
                            coords={"school":list(team_indices.keys())},
                            dims={"strength":["school"]})

In [ ]:
summary_df = az.summary(diff_summary, var_names=['strength']).reset_index()

In [ ]:
import re
def get_team_name(x):
    matches = re.findall("strength\[([0-9]+)]",x)
    assert len(matches) == 1
    team_num = int(matches[0])
    return team_indices_reverse[team_num]

In [ ]:
summary_df['team'] = summary_df['index'].apply(get_team_name)

In [ ]:
summary_df.sort_values("mean", ascending=False).head(10)

# Player model

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [ ]:
url = "https://www.naqt.com/stats/tournament/individuals.jsp?tournament_id=9500&playoffs=true"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
tb = soup.find("table", class_="data-freeze-2")

In [ ]:
all_players = [str(x) for x in tb.find_all("a") if "tournament/team" not in str(x)]
pattern_player_number = "team_member_id=([0-9]+)"
pattern_player_name = ">(.+)<"
player_number_tuple = [
    (re.findall(pattern_player_number, line)[0], re.findall(pattern_player_name, line)[0])
    for line in all_players
]

In [ ]:
result = pd.DataFrame()
for i, (number, player) in enumerate(player_number_tuple):
    if i % 10 == 0:
        print(i)
#     try:
    tmp = pd.read_html(
        "https://www.naqt.com/stats/tournament/player.jsp?team_member_id=" + number
    )[1]
    tmp["team"] = player
    result = pd.concat([result, tmp])
    time.sleep(0.05)
#     except:
#         print("{0} {1} {2}".format(i, number, team))

In [ ]:
result = result.rename({'team':'player'}, axis=1)

In [ ]:
result.info()

In [ ]:
(result
 .query("Round != 'Total'")
 .drop("P%", axis=1)
 .astype({"Round":np.int8,
         "Opponent":'category',
         'Result':'category',
         'TUH':np.int8,
         "P":np.int8,
         "TU":np.int8,
         "I":np.int8,
         "player":"category"
         })

)